# Day 7
## Part 1

Represent the rules as a directed graph, with colours of bags as nodes and the "can be contained by" relationship as edges. Add the weight in case it's useful later on. Then count the number of nodes a search from the "shiny gold" node will find. 

Represent the graph as a dictionary with nodes as keys and edges as tuples of neighbour and weight.

In [21]:
import re
from collections import namedtuple, defaultdict


Edge = namedtuple('Edge', 'nbr wgt')


def parse_data(s):
    g = defaultdict(list)

    for line in s.strip().splitlines():
        container, contained = line.split(" bags contain ")
        for bag in contained.split(", "):
            if m := re.match('(\d+) (.+) bag', bag):
                g[m.group(2)].append(Edge(container, int(m.group(1))))

    return g

In [24]:
test_data = '''light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.'''

test_graph = parse_data(test_data)
test_graph

defaultdict(list,
            {'bright white': [Edge(nbr='light red', wgt=1),
              Edge(nbr='dark orange', wgt=3)],
             'muted yellow': [Edge(nbr='light red', wgt=2),
              Edge(nbr='dark orange', wgt=4)],
             'shiny gold': [Edge(nbr='bright white', wgt=1),
              Edge(nbr='muted yellow', wgt=2)],
             'faded blue': [Edge(nbr='muted yellow', wgt=9),
              Edge(nbr='dark olive', wgt=3),
              Edge(nbr='vibrant plum', wgt=5)],
             'dark olive': [Edge(nbr='shiny gold', wgt=1)],
             'vibrant plum': [Edge(nbr='shiny gold', wgt=2)],
             'dotted black': [Edge(nbr='dark olive', wgt=4),
              Edge(nbr='vibrant plum', wgt=6)]})

Do a depth first search to find every accessible node from "shiny gold".

In [27]:
def part_1(graph):
    # Nodes that have already been seen, track these
    # in case of cycles
    seen = {"shiny gold"}
    result = 0
    # Nodes that are yet to be searched
    search = ["shiny gold"]
    
    while search:
        # DFS means use the most recent addition to
        # the search
        node = search.pop()
        
        for edge in graph[node]:
            if edge.nbr not in seen:
                seen.add(edge.nbr)
                result += 1
                search.append(edge.nbr)
                
    return result

assert part_1(test_graph) == 4

In [28]:
graph = parse_data(open('input').read())
part_1(graph)

296

## Part 2

First need to reverse the graph, so the edge relationship is now "contains" 

In [31]:
def reverse_graph(graph):
    g = defaultdict(list)
    
    for node in graph:
        for edge in graph[node]:
            g[edge.nbr].append(Edge(node, edge.wgt))
            
    return g

test_graph_2 = reverse_graph(test_graph)
test_graph_2

defaultdict(list,
            {'light red': [Edge(nbr='bright white', wgt=1),
              Edge(nbr='muted yellow', wgt=2)],
             'dark orange': [Edge(nbr='bright white', wgt=3),
              Edge(nbr='muted yellow', wgt=4)],
             'bright white': [Edge(nbr='shiny gold', wgt=1)],
             'muted yellow': [Edge(nbr='shiny gold', wgt=2),
              Edge(nbr='faded blue', wgt=9)],
             'dark olive': [Edge(nbr='faded blue', wgt=3),
              Edge(nbr='dotted black', wgt=4)],
             'vibrant plum': [Edge(nbr='faded blue', wgt=5),
              Edge(nbr='dotted black', wgt=6)],
             'shiny gold': [Edge(nbr='dark olive', wgt=1),
              Edge(nbr='vibrant plum', wgt=2)]})

DFS again, but keep track of the multiple of bags e.g a light red bag needs 2 muted yellows, which need 4 shiny golds (2 each). I'm assuming it's acyclic as otherwise there wouldn't be an answer, so there's no need to track which nodes have been visited.

In [36]:
def part_2(graph):
    result = 0
    search = [("shiny gold", 1)]
    
    while search:
        node, multiple = search.pop()
        
        for edge in graph[node]:
            result += multiple * edge.wgt
            search.append((edge.nbr, multiple * edge.wgt))
            
    return result

assert part_2(test_graph_2) == 32

In [38]:
test_data_2 ='''shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.'''

test_graph_3 = reverse_graph(parse_data(test_data_2))
assert part_2(test_graph_3) == 126

In [39]:
part_2(reverse_graph(graph))

9339